We implement the most famous solutions to the differential equation $\frac{\partial f}{\partial t} + r S \frac{\partial f}{\partial S} + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 f}{\partial S^2} = rf$ for the prices of European call and put options. These formulas are $c = S_0 N(d_1) - K e^{-r T} N(d_2)$ for the price of call and $p = Ke^{-rT} N(-d_2) - S_0 N(-d_1)$ for the price of put where 
$d_1 = \frac{\ln(S_0/K) + (r+\sigma^2/2)T}{\sigma \sqrt{T}}$ and $d_2 = \frac{\ln(S_0/K) + (r-\sigma^2/2)T}{\sigma \sqrt{T}} = d_1 - \sigma \sqrt{T}$

In [8]:
import numpy as np
import scipy.stats as ss

In [9]:
#BSM pricer for European option on non-dividend stock
def BlackScholesMerton(option_type, S0, K, r, sigma, T):
    d1 = (np.log(S0/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    if option_type == 'call':
        return S0 * ss.norm.cdf(d1) - K * np.exp(-r*T) * ss.norm.cdf(d2)
    elif option_type == 'put':
        return K * np.exp(-r*T) * ss.norm.cdf(-d2) - S0 * ss.norm.cdf(-d1)

In [10]:
#test the function on example 15.6
print('call price is ', BlackScholesMerton('call', 42, 40, 0.1, 0.2, 0.5))
print('put price is ', BlackScholesMerton('put', 42, 40, 0.1, 0.2, 0.5))

call price is  4.759422392871532
put price is  0.8085993729000922


In [11]:
#BSM pricer for European option on stock with dividend
def BlackScholesMerton_d(d, x1, x2, option_type, S0, K, r, sigma, T): #d is the dividend on each ex-dividend date and x1, x2 were ex-divident date in months
    d_present = d * np.exp(-r * x1 / 12) + d * np.exp(-r * x2 / 12)
    return BlackScholesMerton(option_type, S0 - d_present, K, r, sigma, T)


In [12]:
#test the function on example 15.9
BlackScholesMerton_d(0.5, 2, 5, 'call', 40, 40, 0.09, 0.3, 0.5)

3.671233209047678

In [ ]:
#extension to American option
